In [2]:
%pylab
import time, multiprocessing

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
N = 2**28
x = random.rand(N)

In [4]:
def vsum(x,q):
    q.put(sum(x))
    q.close()
    
def compute_mean(x,np):
    N = len(x)
    jobs = []
    qs = []
    for i in range(np):
        q = multiprocessing.Queue()
        i1 = int(i*N/np)
        i2 = int((i+1)*N/np)
        job = multiprocessing.Process(target=vsum,args=[x[i1:i2],q])
        qs.append(q)
        jobs.append(job)
        
    for job in jobs:
        job.start()
    
    for job in jobs:
        job.join()
    
    s = 0
    for q in qs:
        s += q.get()
        
    return s/N    

In [10]:
%%time

np = 4
avg = compute_mean(x,np)
print("Average = {:12.8f}".format(avg))

Average =   0.49999178
CPU times: user 6.93 ms, sys: 18.3 ms, total: 25.2 ms
Wall time: 444 ms


In [9]:
t = [1.29, 0.667, 0.377, 0.357, 0.337]